In [17]:
import pandas as pd, json, numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [32]:
SC=json.loads(file('../json/SC2.json','r').read())
cnc_path='../../universal/countries/'
cnc=pd.read_excel(cnc_path+'cnc.xlsx').set_index('Name')

In [36]:
for c in SC:
    if len(SC[c])>0:

        print c,
        #read mdf data
        cpath=str(cnc.T.loc[c]['ISO2']).lower()
        if cpath=='nan':cpath='na'
        cpath='../countries/'+cpath
        mdf_dest=pd.read_json(json.loads(file(cpath+'/json/mdf_dest.json','r').read()))
        mdf_arrv=pd.read_json(json.loads(file(cpath+'/json/mdf_arrv.json','r').read()))
        mdf_dest['ID']=mdf_dest['From']
        mdf_arrv['ID']=mdf_arrv['To']
        mdf=pd.concat([mdf_dest,mdf_arrv])

        #save combined - not necessary, space hog
        #file(cpath+"/json/mdf.json",'w').write(json.dumps(mdf.reset_index().to_json()))

        #parse data into flights
        mdg=mdf.set_index(['ID','City','Airport','Airline'])
        flights={}
        minn=1.0 #want to see minimum 1 flight in the past 2 weeks
        for i in mdg.index.get_level_values(0).unique():
            #2 weeks downloaded. want to get weekly freq. but multi by 2 dept+arrv
            d=4.0
            if i not in flights:flights[i]={}
            for j in mdg.loc[i].index.get_level_values(0).unique():
                if len(mdg.loc[i].loc[j])>minn: #minimum 1 flights required in this period at least once every 2 weeks
                    if j not in flights[i]:flights[i][j]={'airports':{},'7freq':0}
                    flights[i][j]['7freq']=len(mdg.loc[i].loc[j])/d 
                    for k in mdg.loc[i].loc[j].index.get_level_values(0).unique():
                        if len(mdg.loc[i].loc[j].loc[k])>minn:
                            if k not in flights[i][j]['airports']:flights[i][j]['airports'][k]={'airlines':{},'7freq':0}
                            flights[i][j]['airports'][k]['7freq']=len(mdg.loc[i].loc[j].loc[k])/d
                            for l in mdg.loc[i].loc[j].loc[k].index.get_level_values(0).unique():
                                try:
                                    if len(mdg.loc[i].loc[j].loc[k].loc[l])>minn: 
                                        if l not in flights[i][j]['airports'][k]['airlines']:flights[i][j]['airports'][k]['airlines'][l]={'7freq':0}
                                        flights[i][j]['airports'][k]['airlines'][l]['7freq']=len(mdg.loc[i].loc[j].loc[k].loc[l])/d
                                except:pass
        file(cpath+"/json/flights.json",'w').write(json.dumps(flights))

Libyan Arab Jamahiriya Canada Turkmenistan Lithuania FYR of Macedonia Cambodia Dem. Rep. of Congo Ethiopia Aruba Sri Lanka Guinea-Bissau Argentina Bolivia Cameroon Burkina Faso Ghana Saudi Arabia Japan Cape Verde Slovenia Guatemala Bosnia and Herzegovina Kuwait Russian Federation Jordan Dominica Liberia Maldives Pakistan Oman Tanzania Greenland Gabon Niue Monaco New Zealand Yemen Jersey Jamaica Albania Samoa United Arab Emirates Kosovo India Azerbaijan Lesotho Saint Vincent and the Grenadines Kenya Tajikistan Turkey Afghanistan Bangladesh Mauritania Solomon Islands Turks and Caicos Islands Saint Lucia Mongolia France Syrian Arab Republic Bermuda Slovakia Somalia Peru Vanuatu Nauru Seychelles Norway Malawi Cook Islands Benin Cuba Montenegro Saint Kitts and Nevis Togo China Armenia Timor-Leste Dominican Republic Ukraine Bahrain Tonga Finland Western Sahara Indonesia Mauritius Sweden Vietnam British Virgin Islands Mali Bulgaria United States Romania Angola Cayman Islands South Africa Cypr